# Quantum-Classical Hybrid Machine Learning

Quantum-classical hybrid machine learning is a kind of NISQ algorithm for machine learning using a quantum computer.  
As the name suggests, it uses a quantum computer and a classical computer hybridly. The basic configuration is shown in the figure below.

<img src="../tutorial-ja/img/250_img.png" width="80%">

The basic idea is the same as VQE and QAOA.  
The quantum computer and the classical computer are run alternately, and the parameter $\theta$ that the quantum circuit has is repeatedly updated to minimize (maximize) the target quantity.
The calculation to find the updated values of parameters is done by the classical computer.   

Classical computer performs the calculations that are possible with classical calculations, and by keeping the parameters as classical data, the quantum circuit can be initialized and executed repeatedly.
This reduces the size of the quantum circuit and makes it suitable for execution on NISQ devices.

VQE and QAOA defined a Hamiltonian for the problem, and updated the parameters of the quantum circuit so that the expected value of the Hamiltonian is minimum (maximum). In contrast, in quantum-classical hybrid machine learning, a loss function that takes the measurement results of the quantum circuit as its argument is defined in the classical computer side. 
As a post-processing step, the classical computer calculates the output of the loss function and updates the parameters to minimize the loss function.  
This eliminates the need to define a Hamiltonian and allows us to directly apply the loss function, which is widely used in machine learning.  

The quantum circuitry that runs in a quantum computer can be divided into three major parts.  

First, in the "Data encoding" layer, the input data for training is embedded into the quantum circuit.  
Next, the "Parametric quantum circuit" layer consists of a combination of two qubit gates to entangle and a rotational gate to update the parameters. It is in this part that the probability distribution of the model is learned by updating parameters.  
Finally, the "Measurement" layer measures the quantum state and passes the measurement results to the classical computer.

Learning the parameters $\theta$ of a quantum circuit is similar to learning the weight parameters in a conventional neural network.  
Therefore, the elements necessary for successful learning are similar to those of neural networks, for example,

1. How to embed data
2. Loss function design
3. Parameter update method
4. Parametric quantum circuit (learning layer) design

A variety of methods have been proposed and used for each of these.  
In the following tutorials, we will show some simple learning examples using concrete models.

### References

Benedetti, Marcello, et al. "Parameterized quantum circuits as machine learning models." Quantum Science and Technology 4.4 (2019): 043001.